In [1]:
import cv2
import torch

# Load the custom-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='model.pt')  # Specify your model path

# Initialize the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()
else:
    print("Webcam successfully opened!")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Run detection
    results = model(frame)

    # Annotate the frame with bounding boxes and labels
    annotated_frame = results.render()[0]  # Get the first frame (since results.render() returns a list)

    # Display the annotated frame
    cv2.imshow("Fruit Detection", annotated_frame)

    # Exit when 'ESC' is pressed
    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-31 Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)

Fusing layers... 
Model summary: 346 layers, 76164816 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 


Webcam successfully opened!


C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\David/.cache\torch\hub\ultralytics_yolov5_master\models

In [ ]:
import cv2
import numpy as np

# Initialize video capture (webcam or video file)
cap = cv2.VideoCapture(0)  # Use 0 for webcam
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Parameters for speed calculation
  # frames per second (adjust accordingly)
pixels_per_meter = 100  # Adjust this based on your camera calibration (experiment with this)

# Variables for tracking
prev_gray = None
prev_points = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect good features to track (corner points)
    if prev_gray is None:
        prev_gray = gray
        prev_points = cv2.goodFeaturesToTrack(gray, mask=None, **{'maxCorners': 100, 'qualityLevel': 0.3, 'minDistance': 7, 'blockSize': 7})

    # Calculate optical flow (tracking points between frames)
    if prev_points is not None:
        next_points, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None)

        # Select good points
        good_new = next_points[status == 1]
        good_old = prev_points[status == 1]

        # Calculate speed if points are found
        if len(good_new) > 0 and len(good_old) > 0:
            # Calculate the distance moved in pixels
            dist_pixels = np.linalg.norm(good_new - good_old, axis=1)
            avg_dist_pixels = np.mean(dist_pixels)  # Average movement

            # Convert to real-world distance (meters)
            distance_meters = avg_dist_pixels / pixels_per_meter

            # Time between frames (seconds)
            time_seconds = 1 / frame_rate

            # Calculate speed in meters per second
            speed_mps = distance_meters / time_seconds

            # Optionally, convert speed to km/h
            speed_kmh = speed_mps * 3.6  # Convert m/s to km/h

            # Display speed on the frame
            cv2.putText(frame, f"Speed: {speed_kmh:.2f} km/h", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw the tracks on the frame
        for new, old in zip(good_new, good_old):
            a, b = new.ravel()
            c, d = old.ravel()
            frame = cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

        # Update previous frame and points
        prev_gray = gray.copy()
        prev_points = good_new.reshape(-1, 1, 2)

    # Display the frame with annotations
    cv2.imshow("Speed Tracking", frame)

    # Exit when 'ESC' is pressed
    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


TypeError: 'NoneType' object is not subscriptable

: 